In [ ]:
pip install -U sentence-transformers

In [ ]:
import pickle

In [ ]:
from sentence_transformers import SentenceTransformer, util
import csv
import torch

In [ ]:
dataset = open('claim_reviews_en.csv')
myReader = csv.reader(dataset)

#Our sentences we like to encode
sentences = []
line = 0
for row in myReader:
  if line != 0:
    sentences.append(row[0])
  line = line + 1

#sentences = sentences[0:10]

#Sentences are encoded by calling model.encode()
#embeddings = model.encode(sentences)

#Print the embeddings
#for sentence, embedding in zip(sentences, embeddings):
#    print("Sentence:", sentence)
#    print("Embedding:", embedding)
#    print("")

# Syntax similarity

## Importations

In [ ]:
import spacy
from spacy import displacy

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


## Variables de travail

In [ ]:
nlp = spacy.load("en_core_web_sm")

dataset = open('claim_reviews_en.csv')
myReader = csv.reader(dataset)

#Our sentences we like to encode
sentences = []
line = 0
for row in myReader:
  if line != 0:
    sentences.append(row[0])
  line = line + 1

tagsList = ["ADJ", "ADP", "ADV", "AUX", "CCONJ", "DET", "INTJ", "NOUN", "NUM", "PART", "PRON", "PROPN", "PUNCT", "SCONJ", "SYM", "VERB", "X"]
pattern = 'Clara Lina Éloïse'

## Fonction pour convertir un POS tag en entier



In [ ]:
def tagToIntetger(tag):
  n = len(tagsList)
  for i in range(n):
    if tag == tagsList[i]:
      return i

## Transformation des phrases en vecteurs de POS tags puis en vecteurs d'entiers

In [ ]:
## Rq: exécution 6'

#sentences = sentences[0:5]
sentencesToken = []

for s in sentences:
  t = []
  doc = nlp(s)
  for token in doc:
    t.append(tagToIntetger(token.pos_))
  sentencesToken.append(t)

patternToken = []
doc = nlp(pattern)
for token in doc:
  patternToken.append(tagToIntetger(token.pos_))

#for i in range(5):
#  print(sentences[i])
#  print(sentencesToken[i])

#print('')
#print(pattern)
#print(patternToken)

## Recherche du pattern dans les phrases

On a un motif de 3 mots que l'on recherche n'importe où dans les phrases.

In [ ]:
strongPattern1 = "Lola said that"
strongPattern2 = 'Lola said "'

patternToken1 = []
doc = nlp(strongPattern1)
for token in doc:
  patternToken1.append(tagToIntetger(token.pos_))

patternToken2 = []
doc = nlp(strongPattern2)
for token in doc:
  patternToken2.append(tagToIntetger(token.pos_))

In [ ]:
index = -1

for v in sentencesToken:
  index = index + 1
  n = len(v)
  for i in range(n-2):
    if v[i] == patternToken[0]:
      if v[i+1] == patternToken[1]:
        if v[i+2] == patternToken[2]:
          #print(nlp(sentences[index])[i:i+3])
          x=0

print('')
index = -1

for v in sentencesToken:
  index = index + 1
  n = len(v)
  for i in range(n-2):
    if v[i] == patternToken1[0]:
      if v[i+1] == patternToken1[1]:
        if v[i+2] == patternToken1[2]:
          print(nlp(sentences[index])[i:i+3])

index = -1

for v in sentencesToken:
  index = index + 1
  n = len(v)
  for i in range(n-2):
    if v[i] == patternToken2[0]:
      if v[i+1] == patternToken2[1]:
        if v[i+2] == patternToken2[2]:
          print(nlp(sentences[index])[i:i+3])

Ici on veut comparer une phrase avec un TAG à la fois.

On va commencer par choisir un TAG et chercher combien sont présents dans la phrase.

In [ ]:
def countTAG(tag, sToken):
  intValue = tagToIntetger(tag)
  nb = 0
  for i in sToken:
    if i == intValue:
      nb = nb+1
  return nb

In [ ]:
def tagsVector(sToken):
  v = []
  for tag in tagsList:
    v.append(countTAG(tag, sToken))
  return v

In [ ]:
def diff(sToken1, sToken2):
  v1, v2 = tagsVector(sToken1), tagsVector(sToken2)
  diff = [abs(v1[i]-v2[i]) for i in range(17)]
  diff[7], diff[15] = 2*diff[7], 2*diff[15]
  dist = 0
  for i in diff:
    dist = dist + i
  dist = float(dist/(len(diff)+2))
  return dist
  

Cherchons les phrases les plus ressemblantes à la première.

In [ ]:
n = len(sentencesToken)
first_sentence = sentencesToken[0]
scores = {sentences[0]:0}
for i in range(1,n):
  scores[sentences[i]] = diff(sentencesToken[i],first_sentence)

scores = sorted(scores.items(), key=lambda t: t[1])


In [ ]:
print(scores[1])

('A blog site publication stated that the President fired his Chief of Staff, further appointing Ajimobi as a replacement.', 0.15789473684210525)


On veut continuer à prendre en compte la position des mots.

On peut découper les phrases en petits pattern et chercher les ressemblances.

On cherche si deux phrases ont des pattern en commun.

In [ ]:
def commonPatterns(s1, s2):
  nb = 0

  n1, n2 = len(s1), len(s2)
  p1, p2 = [], []
  for i in range(n1-2):
    p1.append(s1[i:i+3])
  for i in range(n2-2):
    p2.append(s2[i:i+3])

  n1, n2 = len(p1), len(p2)
  for i in range(n1):
    p = p1[i]
    for k in range(n2):
      if p == p2[k]:
        nb = nb+1
  return nb

In [ ]:
print(commonPatterns(sentencesToken[0], sentencesToken[1]))

2


In [ ]:
n = len(sentencesToken)

for i in range(n):
  for k in range(i+1,n):
    #print("Score between ", sentences[i], " and ", sentences[k], " = ", commonPatterns(sentencesToken[i], sentencesToken[k]))
    score = commonPatterns(sentencesToken[i], sentencesToken[k])
    if score >= 50:
      print("Sentence 1 = ", sentences[i])
      print("Sentence 2 = ", sentences[k])
      print("Score: ", score)
      print("")


In [ ]:
# TESTS

print(sentences[0])
print(sentencesToken[0])

Ratan Tata Says, If The Death Of 65 Soldiers Increases The Popularity Of A Prime Minister, Then Nobody Can Save This Country
[11, 11, 15, 12, 13, 5, 7, 1, 8, 7, 15, 5, 7, 1, 5, 11, 11, 12, 2, 10, 3, 15, 5, 7]


In [ ]:
import pickle

with open ( "pos_similarities_examples.pickle", "rb" ) as f:
    idxs = pickle.load(f)

In [ ]:
print (idxs)
print(len(idxs))

[46304, 26590, 34527, 1086, 8928, 13250, 9224, 10638, 37749, 6698, 25043, 7950, 38097, 24932, 18536, 21229, 16509, 42001, 13103, 1614, 13492, 22699, 7911, 13591, 28630, 33939, 1454, 2370, 12583, 30012, 6524, 1742, 11918, 36433, 24428, 18680, 39172, 9882, 44833, 35635, 14901, 9002, 217, 19501, 12728, 16976, 33281, 36175, 35940, 18690, 1620, 21596, 34292, 34550, 42763, 33625, 18693, 8603, 6343, 14632, 39500, 23843, 15740, 23980, 1250, 46644, 33195, 19205, 34157, 28789, 36324, 6672, 9465, 46784, 5401, 1963, 37092, 23555, 8429, 21604, 8104, 34816, 25117, 23151, 8007, 11268, 36141, 35972, 4395, 27252, 7915, 1980, 17970, 36208, 32851, 15182, 45530, 24986, 5224, 6740]
100


In [ ]:
n = len(sentencesToken)
scoreMatrix = [[] for _ in range(len(idxs))]
line = 0

for i in idxs:
  scoreMatrix[line].append((i,sentences[i],0))
  for k in range(n):
    if k not in idxs:
      score = commonPatterns(sentencesToken[i], sentencesToken[k])
      scoreMatrix[line].append((k,sentences[k],score))
  line = line+1

In [ ]:
print (scoreMatrix[0][0])

('Ratan Tata Says, If The Death Of 65 Soldiers Increases The Popularity Of A Prime Minister, Then Nobody Can Save This Country', 0)


In [ ]:
with open ( "similarity_matrix_cpn.pkl", "wb" ) as f:
    pickle.dump( scoreMatrix, f )